# Data Preprocessing

## Import Libraries


In [ ]:
# Import Libraries

import matplotlib.dates as dates
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
housing = pd.read_csv('../data/AmesHousing.csv', index_col = 0)
real_estate = pd.read_csv('../data/Ames Real Estate Data.csv', index_col = 0)

In [ ]:
housing.shape

In [ ]:
real_estate.shape

In [ ]:
housing.columns

In [ ]:
real_estate.columns

## Dealing with Missing Data


In [ ]:
from IPython.display import display, HTML

cm = sns.light_palette("green",as_cmap=True)
plt.figure(figsize=(14,20))

# Housing Dataframe
plt.subplot(1,2,1)
sns.heatmap(pd.DataFrame(housing.isnull().sum()/housing.shape[0]*100),annot=True,cmap=sns.color_palette("Blues"),
           linewidth=2,linecolor="white")
plt.title("Housing Dataframe NA Values")

# Real Estate Dataframe
plt.subplot(1,2,2)
sns.heatmap(pd.DataFrame(real_estate.isnull().sum()/real_estate.shape[0]*100),annot=True,cmap=sns.color_palette("Reds"),
           linewidth=2,linecolor="white")
plt.title("Real Estate Dataframe NA Values")


## Mutating Columns

In [ ]:
# Create a new column MasVnrArea2 where if MasVnrArea ==0, 0 else 1
housing['MasVnrArea2'] = np.where(housing['Mas Vnr Area']== 0, 0, 1)

In [ ]:
# Create a new colulmn GarageArea2 where the NA values are now 0
housing['GarageArea2'] = housing['Garage Area'].fillna(0)

In [ ]:
# Create a new colulmn PoolArea2 where the NA values are now 0
housing['PoolArea2'] = housing['Pool Area'].fillna(0)

In [ ]:
# replace NA values with 0
housing['2ndFlrSF'] = housing['2nd Flr SF'].fillna(0)

In [ ]:
# Create a new column HalfBath2 and BsmtHalfBath where having a half bathroom = 0.5 for future mutation
housing['HalfBath2'] = np.where(housing['Half Bath']==1, 0.5, 0)
housing['BsmtHalfBath2'] = np.where(housing['Bsmt Half Bath']==1, 0.5, 0)

In [ ]:
# If BsmtFinType1 is GLQ or ALQ, we use the BsmtFinSF1
housing['BasmtFinSF1']=np.where((housing['BsmtFin Type 1']=='ALQ')|(housing['BsmtFin Type 1']=='GLQ'),\
                                 housing['BsmtFin SF 1'],0)

In [ ]:
# If BsmtFinType2 is GLQ or ALQ, we use the BsmtFinSF2
housing['BasmtFinSF2']=np.where((housing['BsmtFin Type 2']=='ALQ')|(housing['BsmtFin Type 2']=='GLQ'),\
                                 housing['BsmtFin SF 2'],0)

In [ ]:
# The Total Square Feet of Basement Finish is BasmtFinSF1 + BasmtFinSF2
housing['BasmtFinSF'] = housing['BasmtFinSF1'] + housing['BasmtFinSF2']

In [ ]:
# Total Living Area is GrLivArea + BasmtFinSF
housing['total_LivArea'] = housing['Gr Liv Area'] + housing['BasmtFinSF']

In [ ]:
# Create num_bathroom
housing['num_bathroom'] = housing['BsmtHalfBath2'] + housing['HalfBath2'] + \
                          housing['Full Bath'] + housing['Bsmt Full Bath']

In [ ]:
# Check the newly created columns
housing[['PID','BasmtFinSF1', 'BasmtFinSF2', 'BasmtFinSF', 'total_LivArea', 'num_bathroom']]

In [ ]:
housing.columns

In [ ]:
housing.shape

# 91 columns (created 10 new columns)

## Merge Data

In [ ]:
# Merge housing and real_estate dataframe
ames_df = pd.merge(housing, real_estate, how = 'left', left_on = ['PID'], right_on = ['MapRefNo'], \
                   suffixes=('', '_y'))
ames_df.drop(ames_df.filter(regex = '_y$').columns.tolist(), axis = 1, inplace = True)

In [ ]:
pd.set_option('display.max_columns', None)
ames_df.head()

In [ ]:
ames_df.shape

In [ ]:
ames_housing = ames_df

#### Create new column - Iowa State University Latitude and Longtitude

In [ ]:
# Create a new column with Iowa State University Latitude and Longtitude
Iowa_state = [] 
Iowa_state_lat = []
Iowa_state_long = []
for index,row in ames_housing.iterrows():
    Iowa_state.append('(42.02704410769817, -93.64641075013807)')
    Iowa_state_lat.append(42.02704410769817)
    Iowa_state_long.append(-93.64641075013807)
    
ames_housing["ISU_lat_long"] = Iowa_state
ames_housing["ISU_lat"] = Iowa_state_lat
ames_housing["ISU_long"] = Iowa_state_long

In [ ]:
ames_housing[['ISU_lat_long', 'ISU_lat', 'ISU_long']]

#### Calculate Distance using Geopy

In [ ]:
# Importing the geodesic module from the library 
from geopy.distance import geodesic 
  
# Loading the lat-long data for Kolkata & Delhi 
kolkata = (22.5726, 88.3639) 
delhi = (28.7041, 77.1025) 
  
# Print the distance calculated in km 
print(geodesic(kolkata, delhi).km) 

## Final Look into the Data

In [ ]:
pd.set_option('display.max_columns', None)
ames_housing.head()

In [ ]:
ames_housing.shape

In [ ]:
ames_housing.isnull().sum()

## Export Data


In [ ]:
ames_housing.to_csv('../data/final_df.csv')